### Custom Exceptions

In [1]:
class TimeoutError(Exception):
    """Timeout exception"""

In [3]:
try:
    raise TimeoutError('timeout error')
except TimeoutError as ex:
    print(ex)

timeout error


In [5]:
import sys

try:
    raise TimeoutError('timeout error')
except:
    ex_type, ex, tb = sys.exc_info()
    print(ex_type, ex, tb)

<class '__main__.TimeoutError'> timeout error <traceback object at 0x7f7ecc06c980>
